##  DATA EXTRACTION

###  1. Move from xml to pandas dataframe

The basic idea of this step is to have a neat tabular representation of the records in DBLP like <b> [ Key , Author , Title , Year ]</b>. In other words, we would need to transform the existing XML (tree based) representation (dblp.xml) into a .csv file (relational/tabular representation). 

To do so, we need to :

   1. Parse the XML 
   2. Find necessary articles and information 
   3. Write to .csv file 
   4. Plot Extraction results
   5. Clean the csv files
   
This notebook contains steps 1 to 4 step 5 will be the subject of the notebook 'DBLP_Cleaning'. Also, the plot is generated in a seperate notebook but just shown here 

### 1.1 Parsing -- Formatting Issues

It should normally be quite straightforward to parse a valid xml file using the ElementTree (ET) (part of the xml standard library in python).

However, the first problem encountered was that the XML in the 'dblp.xml' is not parseable due to the presence of 'named entity' latin-1 encoding, it was meant to be in the 'numeric' latin-1 and thus creates parse errors. 
(<i>cf Section 2 of 'DBLP_Some_Lessons_Learned' (in the Documents folder)</i>)

To overcome this, we extract the required mapping from the <i> Data Type Definition file (dblp.dtd) </i>, replace the strings accordingly and then use it to build the parse tree using ET


In [1]:
dblp_dtd_dict = {"&Agrave;" : "&#192;","&Aacute;" : "&#193;","&Acirc;" : "&#194;","&Atilde;" : "&#195;",
                 "&Auml;" : "&#196;","&Aring;" : "&#197;","&AElig;" : "&#198;","&Ccedil;" : "&#199;",
                 "&Egrave;" : "&#200;","&Eacute;" : "&#201;","&Ecirc;" : "&#202;","&Euml;" : "&#203;",
                 "&Igrave;" : "&#204;","&Iacute;" : "&#205;","&Icirc;" : "&#206;","&Iuml;" : "&#207;",
                 "&ETH;" : "&#208;","&Ntilde;" : "&#209;","&Ograve;" : "&#210;","&Oacute;" : "&#211;",
                 "&Ocirc;" : "&#212;","&Otilde;" : "&#213;","&Ouml;" : "&#214;","&Oslash;" : "&#216;",
                 "&Ugrave;" : "&#217;","&Uacute;" : "&#218;","&Ucirc;" : "&#219;","&Uuml;" : "&#220;",
                 "&Yacute;" : "&#221;","&THORN;" : "&#222;","&szlig;" : "&#223;","&agrave;" : "&#224;",
                 "&aacute;" : "&#225;","&acirc;" : "&#226;","&atilde;" : "&#227;","&auml;" : "&#228;",
                 "&aring;" : "&#229;","&aelig;" : "&#230;","&ccedil;" : "&#231;","&egrave;" : "&#232;",
                 "&eacute;" : "&#233;","&ecirc;" : "&#234;","&euml;" : "&#235;","&igrave;" : "&#236;",
                 "&iacute;" : "&#237;","&icirc;" : "&#238;","&iuml;" : "&#239;","&eth;" : "&#240;",
                 "&ntilde;" : "&#241;","&ograve;" : "&#242;","&oacute;" : "&#243;","&ocirc;" : "&#244;",
                 "&otilde;" : "&#245;","&ouml;" : "&#246;","&oslash;" : "&#248;","&ugrave;" : "&#249;",
                 "&uacute;" : "&#250;","&ucirc;" : "&#251;","&uuml;" : "&#252;","&yacute;" : "&#253;",
                 "&thorn;" : "&#254;","&yuml;" : "&#255;", "&times;": "&#215;", "&micro;" : "&#181;" ,
                 "&reg;" :"&#174;"  }

The following code replaces the special encoding and writes it to the file 'dblp_clean.xml'

In [2]:
dblp_text = open('dblp.xml', 'r').read()

for key in dblp_dtd_dict.keys():
    dblp_text = dblp_text.replace(key, dblp_dtd_dict[key])
    
dblp_clean = open('dblp_clean.xml', 'w')
dblp_clean.write(dblp_text)

2087788233

### 1.2 Parsing --  XML  Element tree



In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import csv
import requests 

In [2]:
dblp_tree = ET.parse('dblp_clean.xml')
dblp_root = dblp_tree.getroot()

### 1.3 Resource Types and Extraction to .csv

There are many different types of resources in the XML tree. The ones that matter for the sake of this project are outlined as follows: 


   1. Articles : Contain articles published in either Journals or Books and Year
   2. Books    : Contain books published, Year , publisher , isbn etc..
   3. Theses   : Masters theses and Phd theses, contain author, title and university.
   4. Persons  : Contain information about the authors, specially aliases they hold!
   5. Proceedings : Information on Editors of conferences, titles , year etc 
   6. Inproceedings : Contain author, title, year and conference.
   7. Incollections : Contain author, title and year.
   
For each record type in DBLP, there are a few that have no Author field, which is inconvenient for our project as we wish to make a recommender system based on Authors and Titles. The proportion retrieved among the ones that do have Author information is 100% for all record types. The unretrieved proportion is shown in the plot below.

#### 1.3.1 Articles 

In [3]:
articles_w_authors  = len(dblp_root.findall('article'))

with open('dblp_articles.csv', 'w', encoding='utf-8') as csvfile:
    cols = ['Article', 'Author', 'Title', 'Publication', 'Year']
    out = csv.DictWriter(csvfile, cols)
    out.writeheader()
    for article in dblp_root.iter('article'):
        if len(article.findall('author')) != 0:
            for author in article.findall('author'):
                try:
                    publication = article.find('journal').text if article.find('journal') != None else article.find('booktitle').text
    
                    out.writerow({'Article': article.get('key'),
                                  'Author': author.text,
                                  'Title': ''.join(article.find('title').itertext()),
                                  'Publication': publication,
                                  'Year': article.find('year').text
                             })
                except:
                    print(article.get('key'))
        else:
            articles_w_authors -= 1 

journals/expert/AlonsoAAM97
journals/expert/AlonsoAAM97
journals/expert/AlonsoAAM97
journals/expert/AlonsoAAM97


As we can see from the above output the only publication that is neither a book nor a journal has the following key: <b><i>journals/expert/AlonsoAAM97</i></b>.

Luckily, in the Documentation folder, we have a paper on how to request the record using the key. (<i>DBLP_XML_Requests</i>) here we check why this one record is misbehaving



In [4]:
print(requests.post('http://dblp.uni-trier.de/rec/xml/journals/expert/AlonsoAAM97').text)

<?xml version="1.0" encoding="US-ASCII"?>
<dblp>
<article key="journals/expert/AlonsoAAM97" publtype="informal publication" mdate="2016-12-15">
<author>Gustavo Alonso</author>
<author>Divyakant Agrawal</author>
<author>Amr El Abbadi</author>
<author>C. Mohan 0001</author>
<title>Functionality and Limitations of Current Workflow Management Systems.</title>
<year>1997</year>
<note>unpublished; submitted to IEEE Expert</note>
<ee>http://www.almaden.ibm.com/cs/exotica/wfmsys.pdf</ee>
</article>
</dblp>



Clearly, It was a paper that was unpublished at the time but submitted to IEEE Expert. therefore, we subtract this record from our 'articles_w_authors' counter

In [5]:
n_retrieved = pd.read_csv('dblp_articles.csv').Article.nunique()

print('Proportion retrieved : ')
print(n_retrieved / (articles_w_authors-1))

print('Proportion of articles with authors in DBLP: ')
print(n_retrieved / len(dblp_root.findall('article')))

Proportion retrieved : 
1.0
Proportion of articles with authors in DBLP: 
0.9945393108183805


#### 1.3.2  Books

In [6]:
books_w_authors = len(dblp_root.findall('book'))

with open('dblp_books.csv', 'w', encoding='utf-8') as csvfile:
    cols = ['Book', 'Author', 'Title', 'Year']
    out = csv.DictWriter(csvfile, cols)
    out.writeheader()
    counter = 0
    for book in dblp_root.iter('book'):
        if len(book.findall('author')) != 0:
            for author in book.findall('author'):
                try:
                    out.writerow({'Book': book.get('key'),
                                  'Author': author.text,
                                  'Title': ''.join(book.find('title').itertext()),
                                  'Year': book.find('year').text
                                })
                except:
                    print(book.get('key'))
        else:
            books_w_authors -= 1


In [7]:
n_retrieved = pd.read_csv('dblp_books.csv').Book.nunique()

print('Proportion retrieved : ')
print(n_retrieved/ books_w_authors)

print('Proportion of books with authors in DBLP: ')
print(n_retrieved / len(dblp_root.findall('book')))

Proportion retrieved : 
1.0
Proportion of books with authors in DBLP: 
0.9089845748958257


#### 1.3.3  Theses

In [59]:
with open('dblp_theses.csv', 'w', encoding='utf-8') as csvfile:
    cols = ['Thesis', 'Author', 'Title', 'Year']
    out = csv.DictWriter(csvfile, cols)
    out.writeheader()
    for thesis in dblp_root.iter('phdthesis'):
        for author in thesis.findall('author'):
            try:
                out.writerow({'Thesis': thesis.get('key'),
                            'Author': author.text,
                            'Title': ''.join(thesis.find('title').itertext()),
                            'Year': thesis.find('year').text
                            })
            except:
                print(thesis.get('key'))
    
    for thesis in dblp_root.iter('mastersthesis'):
        for author in thesis.findall('author'):
            try:
                out.writerow({'Thesis': thesis.get('key'),
                            'Author': author.text,
                            'Title': ''.join(thesis.find('title').itertext()),
                            'Year': thesis.find('year').text
                            })
            except:
                print(thesis.get('key'))
       

In [91]:
n_retrieved = pd.read_csv('dblp_theses.csv').Thesis.nunique()

print('Proportion retrieved : ')
print( n_retrieved/ (len(dblp_root.findall('phdthesis')) + len(dblp_root.findall('mastersthesis'))))

print('Proportion of theses with authors in DBLP: ')
print(n_retrieved / len(dblp_root.findall('mastersthesis')+len(dblp_root.findall('phdthesis')))

Proportion retrieved : 
1.0


The 'homepages' resources have what are called the Person Records (<i>cf Page 7 of 'DBLP_Some_Lessons_Learned'</i>). 

#### 1.3.4 Persons

In [7]:
n_persons = len(dblp_root.findall('www'))

with open('dblp_persons.csv', 'w', encoding='utf-8') as csvfile:
    cols = ['Resource', 'Author']
    out = csv.DictWriter(csvfile, cols)
    out.writeheader()
    for resource in dblp_root.iter('www'):
        if 'homepages' in resource.get('key'):
            for author in resource.findall('author'):
                try:
                    out.writerow({'Resource': resource.get('key'),
                                  'Author': author.text,
                                })
                except:
                    print(resource.get('key'))
        else:
            n_persons -= 1

In [43]:
print('Proportion retrieved : ')
pd.read_csv('dblp_persons.csv').Resource.nunique() / n_persons

Proportion retrieved : 


0.999828689076904

#### 1.3.5 Proceedings

In [95]:
proceedings_w_editor = len(dblp_root.findall('proceedings'))

with open('dblp_proceedings.csv', 'w', encoding='utf-8') as csvfile:
    cols = ['Proceeding', 'Editor', 'Title', 'Year']
    out = csv.DictWriter(csvfile, cols)
    out.writeheader()
    for proceeding in dblp_root.iter('proceedings'):
        if len(proceeding.findall('editor')) !=0 :
            
            for editor in proceeding.findall('editor'):
                    try:
                        out.writerow({'Proceeding': proceeding.get('key'),
                                      'Editor': editor.text,
                                      'Title' : ''.join(proceeding.find('title').itertext()),
                                      'Year' : proceeding.find('year').text
                                    })
                    except:
                        print(proceeding.get('key'))
        else:
            proceedings_w_editor -= 1

In [106]:
n_retrieved = pd.read_csv('dblp_proceedings.csv').Proceeding.nunique()

print('Proportion retrieved : ')
print(n_retrieved/ proceedings_w_editor)

print('Proportion of proceedings with editors in DBLP: ')
print(n_retrieved / len(dblp_root.findall('proceedings')) )

Proportion retrieved : 
1.0
Proportion of proceedings with editors in DBLP: 
0.7727166070276295


#### 1.3.6 Inproceedings

In [101]:
inproceedings_w_authors = len(dblp_root.findall('inproceedings'))

with open('dblp_inproceedings.csv', 'w', encoding='utf-8') as csvfile:
    cols = ['Inproceedings', 'Author', 'Title', 'Publication','Year']
    out = csv.DictWriter(csvfile, cols)
    out.writeheader()
    for article in dblp_root.iter('inproceedings'):
        if len(article.findall('author'))!= 0:
            for author in article.findall('author'):
                try:
                    publication = article.find('journal').text if article.find('journal') != None else article.find('booktitle').text
                
                    out.writerow({'Inproceedings': article.get('key'),
                                  'Author': author.text,
                                  'Title': ''.join(article.find('title').itertext()),
                                  'Publication': publication,
                                  'Year': article.find('year').text
                                 })
                except:
                    print(article.get('key'))
        else:
            inproceedings_w_authors -= 1

In [105]:
print('Proportion retrieved : ')
print(pd.read_csv('dblp_inproceedings.csv').Inproceedings.nunique()/inproceedings_w_authors)
print('Proportion of inproceedings with authors in DBLP: ')
print(pd.read_csv('dblp_inproceedings.csv').Inproceedings.nunique()/len(dblp_root.findall('inproceedings')))

Proportion retrieved : 
1.0
Proportion of inproceedings with authors in DBLP: 
0.9985253170568328


#### 1.3.7 Incollection

In [98]:
collection_w_authors = len(dblp_root.findall('incollection'))

with open('dblp_incollections.csv', 'w', encoding='utf-8') as csvfile:
    cols = ['Incollection', 'Author', 'Title', 'Publication','Year']
    out = csv.DictWriter(csvfile, cols)
    out.writeheader()
    for article in dblp_root.iter('incollection'):
        if len(article.findall('author'))!= 0:
            for author in article.findall('author'):
                try:            
                    out.writerow({'Incollection': article.get('key'),
                                  'Author': author.text,
                                  'Title': ''.join(article.find('title').itertext()),
                                  'Publication': article.find('booktitle').text,
                                  'Year': article.find('year').text
                                 })
                except:
                    print(article.get('key'))
        else:
            collection_w_authors -= 1 

In [104]:
print('Proportion retrieved : ')
print(pd.read_csv('dblp_incollections.csv').Incollection.nunique()/collection_w_authors)

print('Proportion of incollections with authors in DBLP: ')
print(pd.read_csv('dblp_incollections.csv').Incollection.nunique()/len(dblp_root.findall('incollection')))

Proportion retrieved : 
1.0
Proportion of incollections with authors in DBLP: 
0.790610599078341


### 1.4 Extraction Results -- Plot 

![results](Plots/results.png)